# SQL Basics
Before use this notebook, you are encouraged to go over basic SQL syntax in  __[Open Geospatial Solutions](https://geog-414.gishub.org/)__ under DuckDB section. 

This notebook demonstrates a simple workflow: __Find population density for Intermediate Zones in Edinburgh__. 

The purposes are 
* access SIMD and Datazone lookup files
* aggregating the population from Datazones to Intermediate Zones
* join population to Intermediate Zones shapefile
* export as geojson for future use 
  
## Installation

DuckDB is extremely easy to install. Go to __[https://duckdb.org/#quickinstall](https://duckdb.org/#quickinstall)__ and select the programming language you are using. 

Due to some version issues with GeoPandas, I recommend following these steps to create a conda environment for this exercise.
<div class="alert alert-block alert-danger">
<b>Note:</b> conda install GeoPandas before DuckDB and others!
</div>

1. conda create -n "geo_env" python=3.13.1
2. conda activate geo_env
3. conda config --env --add channels conda-forge
4. conda config --env --set channel_priority strict
5. conda install python=3 geopandas
6. conda install jupyter

If you are keen to develop your skill in SQL, worth to try a SQL IDE. We recommend DBeaver which can be installed here __[DBeaver](https://dbeaver.io/)__ (use community version). 
This is not necessary for this class.



## Let's start 

Uncomment the following cell to install the required packages.



In [1]:
#%pip install duckdb duckdb-engine jupysql leafmap

In [2]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql


Connect to jupysql to DuckDB using SQLAlchemy-style connection string. %sql for one line SQL, %%sql for multiline SQL.<br>

One of the advantages of DuckDB is its simplicity.<br>

As you notice here it can be connected to a db which can be stored in your hard drive or an in-memory db. <br>

The former is just a file ending with .db no other overheads. Extremely portable. <br>

The latter stores everything in your memory meaning it is very fast but every time you re-start the kernel you lose the db. <br>

No right or wrong. Choose what your need. <br>


In [3]:
# %sql duckdb:///:memory:
%sql duckdb:///SQL.db

Connecting to 'duckdb:///SQL.db'

Use **'Select * from <file, Table, Dataframe>'** to explore what's inside first.

We now access the SIMD csv file which comes with your SIMD downloads.

You can find the file the Lab9 folder on GitHub. 
Download it to your DuckDB local folder to avoid any http block issue. 

DuckDB uses httpfs extension to direcly access to remote files. 
Try it  when you are free. 

In [4]:
%%sql
Select * from 'Data\simd2020_withinds.csv'


Running query in 'duckdb:///SQL.db'

Data_Zone,Intermediate_Zone,Council_area,Total_population,Working_Age_population,SIMD2020v2_Rank,SIMD_2020v2_Percentile,SIMD2020v2_Vigintile,SIMD2020v2_Decile,SIMD2020v2_Quintile,SIMD2020v2_Income_Domain_Rank,SIMD2020_Employment_Domain_Rank,SIMD2020_Health_Domain_Rank,SIMD2020_Education_Domain_Rank,SIMD2020_Access_Domain_Rank,SIMD2020_Crime_Domain_Rank,SIMD2020_Housing_Domain_Rank,income_rate,income_count,employment_rate,employment_count,CIF,ALCOHOL,DRUG,SMR,DEPRESS,LBWT,EMERG,Attendance,Attainment,no_qualifications,not_participating,University,crime_count,crime_rate,overcrowded_count,nocentralheating_count,overcrowded_rate,nocentralheating_rate,drive_petrol,drive_GP,drive_post,drive_primary,drive_retail,drive_secondary,PT_GP,PT_post,PT_retail,broadband
S01006506,Culter,Aberdeen City,894,580,4691,68,14,7,4,3936.0,3220.0,5174,5887,4724,4664.0,3248.0,8%,71,8%,49,65,29,30,70,13%,0%,74,85%,5.88,53.0,0%,30%,11,125,87,10,10%,1%,2.540103211,3.074294725,1.616238532,2.615747423,1.544260321,9.930833333,8.86358945,5.856135321,6.023405963,11%
S01006507,Culter,Aberdeen City,793,470,4862,70,14,7,4,4829.0,4481.0,5051,4384,2148,4602.0,3486.0,5%,43,5%,25,45,130,126,81,14%,0%,86,85%,5.96,96.0,2%,12%,10,128,85,4,10%,0%,3.91507177,4.309811603,2.555858254,3.64669697,2.8496561,11.04281609,9.978271531,7.515,7.926028708,1%
S01006508,Culter,Aberdeen City,624,461,5686,82,17,9,5,4460.0,5110.0,5942,5915,4200,4563.5,5342.0,6%,40,4%,19,45,71,18,41,13%,4%,69,90%,5.75,39.0,1%,19%,8,130,31,8,5%,1%,3.323024883,3.784548989,1.440991446,3.247325103,2.062255054,10.61676829,8.620699844,4.321493002,5.770909798,1%
S01006509,Culter,Aberdeen City,537,307,4332,63,13,7,4,3481.0,3229.0,3871,6401,3982,5626.0,4394.5,10%,52,8%,26,80,80,28,103,16%,5%,88,94%,6.20,80.0,0%,25%,4,75,42,6,7%,1%,2.622991379,2.778025862,2.620681034,1.936907895,2.160142241,10.03647059,7.935112069,8.433327586,8.329818966,11%
S01006510,Culter,Aberdeen City,663,415,3913,57,12,6,3,3344.0,3448.0,3049,4092,5588,3885.0,3736.0,10%,68,8%,32,95,89,44,139,22%,5%,89,80%,5.87,77.0,6%,16%,11,168,50,7,9%,1%,2.115003882,2.358334627,2.408416149,1.845671642,1.784635093,9.65,5.568963509,6.966428571,6.632608696,0%
S01006511,Culter,Aberdeen City,759,453,6253,90,18,9,5,5469.0,5346.0,5783,5410,4974,6928.0,5924.0,4%,30,4%,17,50,55,0,54,12%,13%,73,90%,5.79,54.0,2%,13%,*,*,27,8,4%,1%,1.516910786,1.769580559,3.227433422,2.517456897,2.593052597,8.614008264,4.934953395,7.66855526,7.340099867,5%
S01006512,Culter,Aberdeen City,539,345,5692,82,17,9,5,6264.0,6206.0,6586,6506,547,4528.0,4815.0,2%,13,2%,8,40,29,0,41,11%,6%,55,93%,5.86,27.0,0%,15%,7,132,27,9,5%,2%,5.365871869,5.776136802,3.909219653,5.374183673,4.277914258,12.2152381,19.01185934,16.25665703,16.96018304,54%
S01006513,"Cults, Bieldside and Milltimber West",Aberdeen City,788,406,6177,89,18,9,5,6572.0,5695.0,5516,6531,1490,6928.0,6394.0,2%,14,3%,13,40,28,0,138,21%,0%,72,94%,6.10,40.0,3%,28%,*,*,15,4,3%,1%,3.573032407,4.322546296,5.524486883,1.945431034,4.910960648,6.262142857,11.26949074,10.98277392,11.09934414,3%
S01006514,"Cults, Bieldside and Milltimber West",Aberdeen City,1123,709,6715,97,20,10,5,6704.0,6658.0,6733,6847,1858,5507.0,6868.0,2%,17,2%,12,25,18,0,107,12%,8%,59,94%,6.50,19.0,1%,22%,9,81,10,3,1%,0%,2.722303452,3.517337009,4.693964525,2.889620253,4.061291946,7.039017857,9.742909875,10.00215724,9.730057526,14%
S01006515,"Cults, Bieldside and Milltimber West",Aberdeen City,816,529,6363,92,19,10,5,6955.0,6803.0,6846,6838,659,6844.0,6174.5,1%,5,1%,7,25,23,0,34,10%,0%,58,96%,6.00,15.0,0%,20%,*,*,29,1,4%,0%,4.444311024,5.177401575,6.337329396,3.063817734,5.78468832,6.878787129,17.18346129,17.09095472,17.02836614,22%


In [5]:
# %%sql

# INSTALL httpfs;
# LOAD httpfs;

In [6]:
# %%sql
# Select * from read_csv('https://your online file.csv');

Similarly, we explore the lookup file.

We can find that **SIMD** file has a **Intermediate_Zone** field but no Intermediate Zone Code. 
Population is at **Data_Zone** level. One **Intermediate_Zone** can have multiple **Data_Zone**.

While the **lookup** file have **Intermediate_Zone** name in **IZ2011_Name** field with additional information such as **IZ2011_Code**, **LA_code**. 

Is there other potential link between these two tables?

In [7]:
%%sql

Select * from 'Data\DataZone2011lookup.csv'; 
        


Running query in 'duckdb:///SQL.db'

DZ2011_Code,DZ2011_Name,IZ2011_Code,IZ2011_Name,LA_Code,LA_Name
S01006506,Culter - 01,S02001236,Culter,S12000033,Aberdeen City
S01006507,Culter - 02,S02001236,Culter,S12000033,Aberdeen City
S01006508,Culter - 03,S02001236,Culter,S12000033,Aberdeen City
S01006509,Culter - 04,S02001236,Culter,S12000033,Aberdeen City
S01006510,Culter - 05,S02001236,Culter,S12000033,Aberdeen City
S01006511,Culter - 06,S02001236,Culter,S12000033,Aberdeen City
S01006512,Culter - 07,S02001236,Culter,S12000033,Aberdeen City
S01006513,"Cults, Bieldside and Milltimber West - 01",S02001237,"Cults, Bieldside and Milltimber West",S12000033,Aberdeen City
S01006514,"Cults, Bieldside and Milltimber West - 02",S02001237,"Cults, Bieldside and Milltimber West",S12000033,Aberdeen City
S01006515,"Cults, Bieldside and Milltimber West - 03",S02001237,"Cults, Bieldside and Milltimber West",S12000033,Aberdeen City


I create copies of these to csvs to my local DuckDB.
The SQL used here is **'Create Table [Table Name] as Select * from <file, Table, Dataframe>'**. 

In [8]:
%%sql 

CREATE TABLE simd AS SELECT * FROM 'Data\simd2020_withinds.csv';
CREATE TABLE lookup AS SELECT * FROM 'Data\DataZone2011lookup.csv';


Running query in 'duckdb:///SQL.db'

Count


After creating the tables in DuckDB. I tried to show them.
Instead of using *Select*, I used **'From [Table Name] limit x'**> 
This is somethine nice not normally used in other DBs. 
Note you can limit number of records you want to see. 

In [9]:
%%sql 

From simd limit 5;

Running query in 'duckdb:///SQL.db'

Data_Zone,Intermediate_Zone,Council_area,Total_population,Working_Age_population,SIMD2020v2_Rank,SIMD_2020v2_Percentile,SIMD2020v2_Vigintile,SIMD2020v2_Decile,SIMD2020v2_Quintile,SIMD2020v2_Income_Domain_Rank,SIMD2020_Employment_Domain_Rank,SIMD2020_Health_Domain_Rank,SIMD2020_Education_Domain_Rank,SIMD2020_Access_Domain_Rank,SIMD2020_Crime_Domain_Rank,SIMD2020_Housing_Domain_Rank,income_rate,income_count,employment_rate,employment_count,CIF,ALCOHOL,DRUG,SMR,DEPRESS,LBWT,EMERG,Attendance,Attainment,no_qualifications,not_participating,University,crime_count,crime_rate,overcrowded_count,nocentralheating_count,overcrowded_rate,nocentralheating_rate,drive_petrol,drive_GP,drive_post,drive_primary,drive_retail,drive_secondary,PT_GP,PT_post,PT_retail,broadband
S01006506,Culter,Aberdeen City,894,580,4691,68,14,7,4,3936.0,3220.0,5174,5887,4724,4664.0,3248.0,8%,71,8%,49,65,29,30,70,13%,0%,74,85%,5.88,53.0,0%,30%,11,125,87,10,10%,1%,2.540103211,3.074294725,1.616238532,2.615747423,1.544260321,9.930833333,8.86358945,5.856135321,6.023405963,11%
S01006507,Culter,Aberdeen City,793,470,4862,70,14,7,4,4829.0,4481.0,5051,4384,2148,4602.0,3486.0,5%,43,5%,25,45,130,126,81,14%,0%,86,85%,5.96,96.0,2%,12%,10,128,85,4,10%,0%,3.91507177,4.309811603,2.555858254,3.64669697,2.8496561,11.04281609,9.978271531,7.515,7.926028708,1%
S01006508,Culter,Aberdeen City,624,461,5686,82,17,9,5,4460.0,5110.0,5942,5915,4200,4563.5,5342.0,6%,40,4%,19,45,71,18,41,13%,4%,69,90%,5.75,39.0,1%,19%,8,130,31,8,5%,1%,3.323024883,3.784548989,1.440991446,3.247325103,2.062255054,10.61676829,8.620699844,4.321493002,5.770909798,1%
S01006509,Culter,Aberdeen City,537,307,4332,63,13,7,4,3481.0,3229.0,3871,6401,3982,5626.0,4394.5,10%,52,8%,26,80,80,28,103,16%,5%,88,94%,6.20,80.0,0%,25%,4,75,42,6,7%,1%,2.622991379,2.778025862,2.620681034,1.936907895,2.160142241,10.03647059,7.935112069,8.433327586,8.329818966,11%
S01006510,Culter,Aberdeen City,663,415,3913,57,12,6,3,3344.0,3448.0,3049,4092,5588,3885.0,3736.0,10%,68,8%,32,95,89,44,139,22%,5%,89,80%,5.87,77.0,6%,16%,11,168,50,7,9%,1%,2.115003882,2.358334627,2.408416149,1.845671642,1.784635093,9.65,5.568963509,6.966428571,6.632608696,0%


In [10]:
%%sql

from lookup limit 3;

Running query in 'duckdb:///SQL.db'

DZ2011_Code,DZ2011_Name,IZ2011_Code,IZ2011_Name,LA_Code,LA_Name
S01006506,Culter - 01,S02001236,Culter,S12000033,Aberdeen City
S01006507,Culter - 02,S02001236,Culter,S12000033,Aberdeen City
S01006508,Culter - 03,S02001236,Culter,S12000033,Aberdeen City


Often, we what to know the scale of the dataset in addition to number of records.
Try to use **Select Distinct [FieldName]  from [TableName]**. 
You can also use **count distinct** to check the total numbers

In [11]:
%%sql

SELECT DISTINCT Intermediate_Zone FROM simd LIMIT 10;

Running query in 'duckdb:///SQL.db'

Intermediate_Zone
Denmore
Newmachar and Fintray
Ellon West
Clashindarroch
New Pitsligo
Peterhead Harbour
Arbroath Cliffburn
Campbeltown
Upper Nithsdale
Georgetown


In [12]:
%%sql

SELECT COUNT(DISTINCT Intermediate_Zone) FROM simd;

Running query in 'duckdb:///SQL.db'

count(DISTINCT Intermediate_Zone)
1250


If you feel this is easy enough, now we are ready to write slightly harder SQLs. They make things a bit complicated in either **Select**, **from**, **where** or add **group by** or sort. </br>
Remember your aggregation functions normally happen within **Select**, such as Sum, Avg. </br>
**where** clause is used for filtering. Notice we use wildcard instead of an exact match for strings. </br>
It has to be worked together with **group by** where your study unit becomes larger, such as from Datazone to Intermediate zones. </br>
Adjust **order by** to sort the results if needed.</br>

The following SQL calculates the sum of the population, originally at the Datazone level, to Intermediate Zones in Edinburgh.


In [13]:
%%sql

SELECT SUM(Total_population) as pop,Intermediate_Zone, Council_area FROM simd where Council_area like '%Edinburgh' group by Intermediate_Zone,Council_area order by pop DESC, Council_area ASC;

Running query in 'duckdb:///SQL.db'

pop,Intermediate_Zone,Council_area
8380,"Canongate, Southside and Dumbiedykes",City of Edinburgh
7821,Dalry and Fountainbridge,City of Edinburgh
7583,"Dalmeny, Kirkliston and Newbridge",City of Edinburgh
7504,Meadows and Southside,City of Edinburgh
6891,Deans Village,City of Edinburgh
6852,"Blackford, West Mains and Mayfield Road",City of Edinburgh
6520,"Old Town, Princes Street and Leith Street",City of Edinburgh
6367,Tollcross,City of Edinburgh
6226,Currie West,City of Edinburgh
6187,Clermiston and Drumbrae,City of Edinburgh


This structure remains even if we add in more conditions. 


In [14]:
%%sql

SELECT SUM(Total_population) as pop,Intermediate_Zone, Council_area FROM simd where Council_area like '%Edinburgh' Or Council_area like '%Glasgow%' group by Intermediate_Zone,Council_area having Sum(Total_population)>5000 order by pop ASC, Council_area ASC; 

Running query in 'duckdb:///SQL.db'

pop,Intermediate_Zone,Council_area
5007,The Inch,City of Edinburgh
5028,Craiglockhart,City of Edinburgh
5039,Glenwood South,Glasgow City
5056,Pollokshields East,Glasgow City
5081,Mosspark,Glasgow City
5082,Battlefield,Glasgow City
5085,Merchiston and Greenhill,City of Edinburgh
5105,Slateford and Chesser,City of Edinburgh
5125,Anderston,Glasgow City
5135,Marchmont East and Sciennes,City of Edinburgh


It is fine to look at the selection results but it would be also useful to create a table out from it for further use. </br>
We add **Create Table [new table name] as**. </br>
In more complicated queries known as transactions, you can create table views rather than tables.  </br>
For DuckDB, we can create very big tables without too much concern about the storage due to its unique structure.  </br>

In [15]:
%%sql

create table if not exists Edinburgh_pop as (
SELECT SUM(Total_population) as pop,Intermediate_Zone, Council_area FROM simd where Council_area like '%Edinburgh' group by Intermediate_Zone,Council_area order by pop DESC, Council_area ASC
);

Running query in 'duckdb:///SQL.db'

Count


You can check out if the new table is created successively. </br>
Also you can view its details. 

In [16]:
%%sql SHOW TABLES;



Running query in 'duckdb:///SQL.db'

name


In [17]:
%%sql

select * from Edinburgh_pop limit 3;

Running query in 'duckdb:///SQL.db'

pop,Intermediate_Zone,Council_area
8380,"Canongate, Southside and Dumbiedykes",City of Edinburgh
7821,Dalry and Fountainbridge,City of Edinburgh
7583,"Dalmeny, Kirkliston and Newbridge",City of Edinburgh


After aggregating the population to Intermediate Zones, you can join it to the lookup file where we have additional Intermediate Zone code information. </br>
The syntax here is **Join** clause after **from**. Note that you need to specify the matching fields between the two tables. </br>
If the matching fields are specifically created to set up *relationship* between tables also maintain *integrity*, they are called *Keys* or specifically *Foreign keys*. </br>
In our case, we just use Intermediate Zone Names. They are not set up as *Keys* but function the same.</br>

Meanwhile, we can see show different Join method works. Change **Left Join** to **Right Join**, **Inner Join** and **Full Join** to see the difference. 


In [18]:
%%sql

With Edinburgh_pop_IZCode as(
    SELECT pop, Edinburgh_pop."Intermediate_Zone", Edinburgh_pop."Council_area", lookup."IZ2011_Code" FROM Edinburgh_pop Left JOIN lookup ON Edinburgh_pop.Intermediate_Zone = lookup."IZ2011_Name"
)
select count(distinct IZ2011_Code) from Edinburgh_pop_IZCode
;

Running query in 'duckdb:///SQL.db'

count(DISTINCT IZ2011_Code)
113


We have got it worked! </br>
But there is still a problem. </br>

Edinburgh has 111 Intermediate Zones in total. But our SQL returns 113 instead.</br>

In your exercises, you will fix the SQL to retrieve correct Intermediate Zones and try to explain the issue. 

Hint: Check matching between Intermdiate Zone Names in SIMD and Lookup table.